<p>
  <b style="font-size: 35px; color: yellow;">Algoritmo de Classificação para Manutenção Preditiva de Falhas </b> <br>
  <b style="font-size: 30px;">Autor:</b> <span style="font-size: 30px;">Paulo Roberto Garcia Junior</span> <br>
  <span style="font-size: 25px;">Engenheiro de Automação e Controle</span> <br>
</p>

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️Instalações e Bibliotecas Necessárias</b> 
</p>

In [ ]:
#Instalações
!pip install xgboost lightgbm
!pip install imbalanced-learn

# Bibliotecas e Módulos Usados
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    roc_curve,
    classification_report,
    confusion_matrix
)
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline


<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Pré-Processamento Inicial e Codificação de Variáveis</b> 
</p>

In [ ]:
# Define o nome do arquivo de entrada (dados brutos)
arquivo_entrada = 'predictive_maintenance.csv' 
# Define o nome do arquivo de saída (dados limpos e transformados)
arquivo_saida = 'predictive_maintenance_final.csv'

try:
    # CARREGAMENTO E SELEÇÃO DE DADOS
    # Carrega o arquivo CSV usando ponto e vírgula (;) como separador.
    df = pd.read_csv(arquivo_entrada, sep=';')
    # Seleciona todas as colunas a partir da terceira (índice 2),
    # removendo colunas de ID ou timestamp irrelevantes.
    df_modificado = df.iloc[:, 2:]

    print("--- DataFrame ANTES das transformações ---")
    print(df_modificado.head())

    # Dicionário para mapear a coluna categórica 'Type' (L, M, H) para valores ordinais (1, 2, 3).
    mapeamento_ordinal = {'L': 1, 'M': 2, 'H': 3}
    # Aplica o mapeamento à coluna 'Type'.
    df_modificado['Type'] = df_modificado['Type'].map(mapeamento_ordinal)
    # Renomeia a coluna para indicar que a codificação foi aplicada, melhorando a clareza.
    df_modificado.rename(columns={'Type': 'Type_Encoded'}, inplace=True)

    # LIMPEZA DE DADOS: CORREÇÃO DE SEPARADOR DECIMAL
    print("\n--- Corrigindo separadores decimais (',' -> '.') ---")
    colunas_para_ignorar = ['Type_Encoded', 'Target', 'Failure Type']

    for coluna in df_modificado.columns:
        if coluna not in colunas_para_ignorar:
            # Converte a coluna para string, substitui ',' por '.', e reconverte para float (número decimal).
            df_modificado[coluna] = df_modificado[coluna].astype(str).str.replace(',', '.').astype(float)
    
    print("Correção concluída.")
    # Salva o DataFrame limpo em um novo arquivo CSV.
    df_modificado.to_csv(arquivo_saida, index=False, sep=';')

    print("\n--- DataFrame DEPOIS de TODAS as transformações ---")
    print(df_modificado.head())
    print("\nVerificando os tipos de dados finais:")
    print(df_modificado.info()) 

    print(f"\n✅ Processo concluído! Arquivo salvo como '{arquivo_saida}'")
    print("A coluna 'Type' foi codificada e os separadores decimais foram corrigidos para '.'.")

#TRATAMENTO DE ERROS
except FileNotFoundError:
    print(f"Erro: O arquivo '{arquivo_entrada}' não foi encontrado. Verifique o nome e o local do arquivo.")
except KeyError as e:
    print(f"Erro: A coluna '{e}' não foi encontrada no dataframe. Verifique se o nome da coluna está correto.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Separação de Features e Target e Divisão em Conjuntos de Treino e Teste</b> 
</p>

In [ ]:
# Define o nome do arquivo de entrada, que é o resultado do pré-processamento anterior.
arquivo_entrada = 'predictive_maintenance_final.csv'

try:
    # CARREGAMENTO E SEPARAÇÃO DE FEATURES E TARGET
    df = pd.read_csv(arquivo_entrada, sep=';')
    coluna_alvo = 'Target'

    # A coluna 'Failure Type' é removida pois ela detalha o tipo da falha, mas o modelo deve prever 'Target' (se há falha ou não).
    X = df.drop(columns=[coluna_alvo, 'Failure Type'])
    # Separa o Target (y): Contém apenas a coluna alvo 'Target'.
    y = df[coluna_alvo]

    print("--- 1. Dados carregados e separados em X e y ---")
    print("Features (pistas) que serão usadas pelo modelo:")
    print(X.columns.tolist())

    # DIVISÃO EM TREINO E TESTE (80/20)
    # X_train, y_train: Para treinar o modelo (80% dos dados).
    # X_test, y_test: Para testar e avaliar o desempenho (20% dos dados).
    # test_size=0.2: Define 20% dos dados para teste.
    # random_state=42: Garante que a divisão seja a mesma sempre que o código for executado (reprodutibilidade).
    # stratify=y: É CRUCIAL para dados desbalanceados. Garante que a proporção da classe alvo (y) seja mantida 
    #              tanto no conjunto de treino quanto no de teste. 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print("\n--- 2. Dados divididos em Treino e Teste ---")
    print(f"Tamanho do conjunto de Treino: {len(X_train)} linhas")
    print(f"Tamanho do conjunto de Teste:  {len(X_test)} linhas")
    print("\nA proporção de falhas foi mantida em ambos os conjuntos.")

# TRATAMENTO DE ERROS
except FileNotFoundError:
    print(f"ERRO: O arquivo '{arquivo_entrada}' não foi encontrado.")
except KeyError as e:
    print(f"ERRO: Uma coluna esperada ({e}) não foi encontrada no arquivo. Verifique o script de preparação.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Escalonamento de Dados, Treinamento e Avaliação do Modelo Base (Random Forest)</b> 
</p>

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("--- 3. Dados de treino e teste escalonados com sucesso! ---")

modelo_base = RandomForestClassifier(n_estimators=100, random_state=42)

print("\n--- 4. Treinando o Modelo Base... ---")

modelo_base.fit(X_train_scaled, y_train)
print("Modelo Base treinado com sucesso!")

print("\n--- 5. Avaliando o desempenho do Modelo Base nos dados de Teste ---")

y_pred_base = modelo_base.predict(X_test_scaled)
# PASSO 1: Calcular as probabilidades para o cálculo da AUC
y_proba_base = modelo_base.predict_proba(X_test_scaled)[:, 1]

# PASSO 2: Calcular a AUC
auc_base = roc_auc_score(y_test, y_proba_base)

print(f"\nAcurácia: {accuracy_score(y_test, y_pred_base):.4f}")
# PASSO 3: Imprimir o valor da AUC
print(f"AUC (Area Under the Curve): {auc_base:.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred_base, target_names=['Sem Falha (0)', 'Falha (1)']))
print("Matriz de Confusão:")
cm = confusion_matrix(y_test, y_pred_base)
print(cm)

#plot da Matriz de Confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
             xticklabels=['Previsto: Sem Falha', 'Previsto: Falha'], 
             yticklabels=['Real: Sem Falha', 'Real: Falha'])
plt.title('Matriz de Confusão - Modelo Base')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')
plt.show()

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Treinamento e Avaliação do Modelo Random Forest com class_weight</b> 
</p>

In [ ]:
print("--- 6. Treinando o Modelo Balanceado... ---")

# O parâmetro class_weight='balanced' ajusta o peso das classes
modelo_balanceado = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

modelo_balanceado.fit(X_train_scaled, y_train)

print("Modelo Balanceado treinado com sucesso!")

print("\n--- 7. Avaliando o desempenho do Modelo Balanceado ---")
y_pred_balanceado = modelo_balanceado.predict(X_test_scaled)

# Cálculo da AUC requer as probabilidades
y_proba_balanceado = modelo_balanceado.predict_proba(X_test_scaled)[:, 1]

# Calculando Acurácia e AUC
acuracia_balanceado = accuracy_score(y_test, y_pred_balanceado)
auc_balanceado = roc_auc_score(y_test, y_proba_balanceado)

# Imprimindo as métricas
print(f"\nAcurácia (Modelo Balanceado): {acuracia_balanceado:.4f}")
print(f"AUC (Area Under the Curve, Modelo Balanceado): {auc_balanceado:.4f}")

print("\nRelatório de Classificação (Modelo Balanceado):")
print(classification_report(y_test, y_pred_balanceado, target_names=['Sem Falha (0)', 'Falha (1)']))

print("Matriz de Confusão (Modelo Balanceado):")
cm_balanceado = confusion_matrix(y_test, y_pred_balanceado)
print(cm_balanceado)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_balanceado, annot=True, fmt='d', cmap='Greens',
             xticklabels=['Previsto: Sem Falha', 'Previsto: Falha'], 
             yticklabels=['Real: Sem Falha', 'Real: Falha'])
plt.title('Matriz de Confusão - Modelo Balanceado')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')
plt.show()

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Implementação e Avaliação do Pipeline com SMOTE e Random Forest</b> 
</p>

In [ ]:
print("--- 8. Criando um pipeline com SMOTE e RandomForest ---")

over_sampler = SMOTE(random_state=42)
modelo_final = RandomForestClassifier(n_estimators=100, random_state=42)

pipeline_smote = Pipeline([
    ('oversampling', over_sampler),
    ('model', modelo_final)
])

print("Treinando o Modelo Final com SMOTE...")
pipeline_smote.fit(X_train_scaled, y_train)
print("Modelo Final treinado com sucesso!")

print("\n--- 9. Avaliando o desempenho do Modelo Final ---")
y_pred_smote = pipeline_smote.predict(X_test_scaled)

# Cálculo da AUC requer as probabilidades, obtidas através do pipeline
y_proba_smote = pipeline_smote.predict_proba(X_test_scaled)[:, 1]

# Calculando Acurácia e AUC
acuracia_smote = accuracy_score(y_test, y_pred_smote)
auc_smote = roc_auc_score(y_test, y_proba_smote)

# Imprimindo as métricas
print(f"\nAcurácia (Modelo com SMOTE): {acuracia_smote:.4f}")
print(f"AUC (Area Under the Curve, Modelo com SMOTE): {auc_smote:.4f}")

print("\nRelatório de Classificação (Modelo com SMOTE):")
print(classification_report(y_test, y_pred_smote, target_names=['Sem Falha (0)', 'Falha (1)']))

print("Matriz de Confusão (Modelo com SMOTE):")
cm_smote = confusion_matrix(y_test, y_pred_smote)
print(cm_smote)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_smote, annot=True, fmt='d', cmap='Reds',
             xticklabels=['Previsto: Sem Falha', 'Previsto: Falha'], 
             yticklabels=['Real: Sem Falha', 'Real: Falha'])
plt.title('Matriz de Confusão - Modelo com SMOTE')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')
plt.show()

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Otimização de Hiperparâmetros com Grid Search e Avaliação do Modelo Otimizado (Random Forest)</b> 
</p>

In [ ]:
print("--- 10. Ajuste de Hiperparâmetros com GridSearchCV ---")

# (O código do pipeline e da configuração do GridSearchCV permanece o mesmo)
pipeline_smote_for_tuning = Pipeline([
    ('oversampling', SMOTE(random_state=42)),
    ('model', RandomForestClassifier(random_state=42)) 
])

param_grid = {
    'model__n_estimators': [100, 200],
    'model__max_depth': [10, 20, None],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=pipeline_smote_for_tuning, 
                           param_grid=param_grid, 
                           cv=5, 
                           scoring='f1', 
                           n_jobs=-1)

print("Iniciando a busca em grade... Isso pode levar alguns minutos.")
grid_search.fit(X_train_scaled, y_train)

print("\nBusca em grade finalizada!")
print(f"Melhor pontuação F1 (em validação cruzada): {grid_search.best_score_:.4f}")
print("Melhores hiperparâmetros encontrados:")
print(grid_search.best_params_)


# --- AVALIAÇÃO FINAL DO MODELO OTIMIZADO (COM MÉTRICAS COMPLETAS) ---
print("\n--- 11. Avaliando o melhor modelo encontrado no conjunto de teste ---")

best_model_rf = grid_search.best_estimator_

# Previsões finais (0 ou 1)
y_pred_best_rf = best_model_rf.predict(X_test_scaled)

# NOVO: Para calcular a AUC, precisamos das probabilidades da classe positiva (Falha)
y_proba_best_rf = best_model_rf.predict_proba(X_test_scaled)[:, 1]

# NOVO: Exibindo as métricas adicionais
print(f"\nAcurácia (RandomForest Otimizado): {accuracy_score(y_test, y_pred_best_rf):.4f}")
print(f"AUC (RandomForest Otimizado): {roc_auc_score(y_test, y_proba_best_rf):.4f}")

# Relatório de Classificação (como antes)
print("\nRelatório de Classificação (RandomForest Otimizado):")
print(classification_report(y_test, y_pred_best_rf, target_names=['Sem Falha (0)', 'Falha (1)']))

# Matriz de Confusão (como antes)
print("Matriz de Confusão (RandomForest Otimizado):")
cm_best_rf = confusion_matrix(y_test, y_pred_best_rf)

# Gráfico da Matriz de Confusão (como antes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm_best_rf, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Previsto: Sem Falha', 'Previsto: Falha'], 
            yticklabels=['Real: Sem Falha', 'Real: Falha'])
plt.title('Matriz de Confusão - RandomForest Otimizado')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')
plt.show()

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Treinamento, Avaliação e Comparação Gráfica do Modelo XGBoost (com SMOTE)</b> 
</p>

In [ ]:
# Importações necessárias para todos os modelos e métricas
print("--- 12. Treinando e Avaliando o XGBoost ---")

pipeline_xgb = Pipeline([
    ('oversampling', SMOTE(random_state=42)),
    ('model', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'))
])

pipeline_xgb.fit(X_train_scaled, y_train)
# Fazendo as previsões e obtendo as probabilidades

y_pred_xgb = pipeline_xgb.predict(X_test_scaled)
y_proba_xgb = pipeline_xgb.predict_proba(X_test_scaled)[:, 1]
# Exibindo as métricas

print(f"\nAcurácia (XGBoost): {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"AUC (XGBoost): {roc_auc_score(y_test, y_proba_xgb):.4f}")
print("\nRelatório de Classificação (XGBoost com SMOTE):")
print(classification_report(y_test, y_pred_xgb, target_names=['Sem Falha (0)', 'Falha (1)']))
print("Matriz de Confusão (XGBoost com SMOTE):")
print(confusion_matrix(y_test, y_pred_xgb))

fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_best_rf)
auc_rf = roc_auc_score(y_test, y_proba_best_rf)
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_proba_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

plt.figure(figsize=(10, 8))
plt.plot(fpr_rf, tpr_rf, label=f'RandomForest Otimizado (AUC = {auc_rf:.4f})', color='orange')
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {auc_xgb:.4f})', color='blue')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label='Modelo Aleatório')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos (Recall)')
plt.title('Comparação da Curva ROC entre Modelos Finais')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

cm_xgb = confusion_matrix(y_test, y_pred_xgb)

print("Matriz de Confusão (XGBoost com SMOTE):")
print(cm_xgb)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Previsto: Sem Falha (0)', 'Previsto: Falha (1)'], 
            yticklabels=['Real: Sem Falha (0)', 'Real: Falha (1)'])
plt.title('Matriz de Confusão - XGBoost com SMOTE')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')
plt.show()


<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Treinamento e Avaliação dos Modelos SVM (Kernel Linear e RBF) </b> 
</p>

In [ ]:
# --- 15. TREINANDO E AVALIANDO O MODELO SVM (KERNEL LINEAR) ---
print("--- 15. Treinando e Avaliando o SVM com Kernel Linear ---")

# Criamos o pipeline com SMOTE e o SVC com kernel='linear'
# probability=True é necessário para calcular a AUC, mas torna o treino mais lento.
pipeline_svm_linear = Pipeline([
    ('oversampling', SMOTE(random_state=42)),
    ('model', SVC(kernel='linear', random_state=42, probability=True))
])

# Treinamos o pipeline
# ASSUMIR: X_train_scaled e y_train estão definidos
pipeline_svm_linear.fit(X_train_scaled, y_train)

# Avaliamos no conjunto de teste
# ASSUMIR: X_test_scaled e y_test estão definidos
y_pred_svm_linear = pipeline_svm_linear.predict(X_test_scaled)
y_proba_svm_linear = pipeline_svm_linear.predict_proba(X_test_scaled)[:, 1]

# Métricas
print(f"\nAcurácia (SVM Linear): {accuracy_score(y_test, y_pred_svm_linear):.4f}")
print(f"AUC (SVM Linear): {roc_auc_score(y_test, y_proba_svm_linear):.4f}")
print("\nRelatório de Classificação (SVM Linear):")
print(classification_report(y_test, y_pred_svm_linear, target_names=['Sem Falha (0)', 'Falha (1)']))
cm_svm_linear = confusion_matrix(y_test, y_pred_svm_linear)
print("Matriz de Confusão (SVM Linear):")
print(cm_svm_linear)


# --- 16. TREINANDO E AVALIANDO O MODELO SVM (KERNEL RBF) ---

print("\n\n--- 16. Treinando e Avaliando o SVM com Kernel RBF ---")
print("(Este modelo pode demorar um pouco mais para treinar...)")

# Criamos o pipeline com SMOTE e o SVC com kernel='rbf' (o padrão)
pipeline_svm_rbf = Pipeline([
    ('oversampling', SMOTE(random_state=42)),
    ('model', SVC(kernel='rbf', random_state=42, probability=True))
])

# Treinamos o pipeline
pipeline_svm_rbf.fit(X_train_scaled, y_train)

# Avaliamos no conjunto de teste
y_pred_svm_rbf = pipeline_svm_rbf.predict(X_test_scaled)
y_proba_svm_rbf = pipeline_svm_rbf.predict_proba(X_test_scaled)[:, 1]

# Métricas
print(f"\nAcurácia (SVM RBF): {accuracy_score(y_test, y_pred_svm_rbf):.4f}")
print(f"AUC (SVM RBF): {roc_auc_score(y_test, y_proba_svm_rbf):.4f}")
print("\nRelatório de Classificação (SVM RBF):")
print(classification_report(y_test, y_pred_svm_rbf, target_names=['Sem Falha (0)', 'Falha (1)']))
cm_svm_rbf = confusion_matrix(y_test, y_pred_svm_rbf)
print("Matriz de Confusão (SVM RBF):")
print(cm_svm_rbf)


# --- 17. PLOTAGEM DAS MATRIZES DE CONFUSÃO (SVMs) ---

print("\n\n--- 17. Plotando as Matrizes de Confusão para SVM Linear e SVM RBF ---")

plt.figure(figsize=(14, 6))

# 1. Matriz de Confusão para SVM Linear
plt.subplot(1, 2, 1) # Cria uma grade 1 linha, 2 colunas, e seleciona a 1ª posição
sns.heatmap(cm_svm_linear, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Previsto: Sem Falha (0)', 'Previsto: Falha (1)'], 
            yticklabels=['Real: Sem Falha (0)', 'Real: Falha (1)'])
plt.title('Matriz de Confusão: SVM Linear')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')

# 2. Matriz de Confusão para SVM RBF
plt.subplot(1, 2, 2) # Seleciona a 2ª posição
sns.heatmap(cm_svm_rbf, annot=True, fmt='d', cmap='Reds',
            xticklabels=['Previsto: Sem Falha (0)', 'Previsto: Falha (1)'], 
            yticklabels=['Real: Sem Falha (0)', 'Real: Falha (1)'])
plt.title('Matriz de Confusão: SVM RBF')
plt.ylabel('Valor Real')
plt.xlabel('Valor Previsto')

plt.tight_layout() # Ajusta automaticamente os parâmetros do subplot
plt.show()

<p style="font-family: 'Times New Roman', Times, serif; font-size: 40px;">
  <b>🏷️ Visualização da Curva ROC para Comparação Final de Todos os Modelos</b> 
</p>

In [ ]:
# --- 17. GRÁFICO FINAL: COMPARANDO TODOS OS MODELOS ---
print("--- 17. Gerando Gráfico de Comparação Final da Curva ROC ---")

# (Certifique-se que as probabilidades de todos os modelos estão carregadas na memória)
# y_proba_best_rf, y_proba_xgb, y_proba_lgbm, y_proba_svm_linear, y_proba_svm_rbf

# Calculando os dados da curva para todos os modelos
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_best_rf)
auc_rf = roc_auc_score(y_test, y_proba_best_rf)

fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_proba_xgb)
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

fpr_svm_linear, tpr_svm_linear, _ = roc_curve(y_test, y_proba_svm_linear)
auc_svm_linear = roc_auc_score(y_test, y_proba_svm_linear)

fpr_svm_rbf, tpr_svm_rbf, _ = roc_curve(y_test, y_proba_svm_rbf)
auc_svm_rbf = roc_auc_score(y_test, y_proba_svm_rbf)

# Configurações de Estilo
plt.figure(figsize=(14, 10))
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams.update({'font.size': 12})

# Plotando todas as curvas
plt.plot(fpr_rf, tpr_rf, label=f'RandomForest Otimizado (AUC = {auc_rf:.4f})', 
         color='darkorange', linewidth=2)
         
plt.plot(fpr_xgb, tpr_xgb, label=f'XGBoost (AUC = {auc_xgb:.4f})', 
         color='dodgerblue', linewidth=2)
         
         
plt.plot(fpr_svm_linear, tpr_svm_linear, label=f'SVM Linear (AUC = {auc_svm_linear:.4f})', 
         color='cyan', linewidth=2, linestyle=':')

plt.plot(fpr_svm_rbf, tpr_svm_rbf, label=f'SVM RBF (AUC = {auc_svm_rbf:.4f})', 
         color='red', linewidth=2, linestyle='--') # Destaque para o campeão de Recall

plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle=':', label='Modelo Aleatório (AUC = 0.50)')

# Finalizando o gráfico
plt.xlim([-0.02, 1.02])
plt.ylim([-0.02, 1.02])
plt.xlabel('Taxa de Falsos Positivos', fontsize=14, labelpad=10)
plt.ylabel('Taxa de Verdadeiros Positivos (Recall)', fontsize=14, labelpad=10)
plt.title('Comparação Final da Curva ROC entre Todos os Modelos', fontsize=18, pad=20)
plt.legend(loc="lower right", fontsize=12, frameon=True, shadow=True, borderpad=1)
plt.grid(True, linestyle=':', alpha=0.7)
plt.tight_layout()
plt.show()